In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
# Add modules to the path
import sys
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\freely_moving_photometry_analysis' )

import matplotlib
import copy
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
import pandas as pd
import pickle
from scipy.signal import decimate
from utils.plotting import HeatMapParams
from utils.plotting import heat_map_and_mean, plot_one_side, calculate_error_bars
from utils.plotting import get_photometry_around_event
from scipy import stats
from utils.individual_trial_analysis_utils import ZScoredTraces, SessionData, CueAlignedData, CueAlignedSidedData, ChoiceAlignedData

In [2]:
def get_all_experimental_records():
    experiment_record = pd.read_csv('T:\\photometry_2AC\\experimental_record.csv', dtype='str')
    experiment_record['date'] = experiment_record['date'].astype(str)
    return experiment_record

In [3]:
def open_experiment(experiment_to_add):
    for index, experiment in experiment_to_add.iterrows():
        #saving_folder = 'S:\\users\\Yvonne\\photometry_2AC\\processed_data\\' + experiment['mouse_id'] + '\\'
        saving_folder = 'T:\\photometry_2AC\\processed_data\\' + experiment['mouse_id'] + '\\'
        #saving_folder = 'C:\\Users\\francescag\\Documents\\PhD_Project\\SNL_photo_photometry\\processed_data' + experiment['mouse_id']  + '\\'
        restructured_data_filename = experiment['mouse_id'] + '_' + experiment['date'] + '_' + 'restructured_data.pkl'
        trial_data = pd.read_pickle(saving_folder + restructured_data_filename)
        dff_trace_filename = experiment['mouse_id'] + '_' + experiment['date'] + '_' + 'smoothed_signal.npy'
        dff = np.load(saving_folder + dff_trace_filename)
        session_traces = SessionData(experiment['fiber_side'], experiment['recording_site'], experiment['mouse_id'], experiment['date'])
    return session_traces

In [4]:
class CustomAlignedData(object):
    def __init__(self, session_data, params):
        #saving_folder = 'S:\\users\\Yvonne\\photometry_2AC\\processed_data\\' + session_data.mouse + '\\'
        saving_folder = 'T:\\photometry_2AC\\processed_data\\' + session_data.mouse + '\\'
        #saving_folder = 'C:\\Users\\francescag\\Documents\\PhD_Project\\SNL_photo_photometry\\processed_data' + session_data.mouse + '\\'
        restructured_data_filename = session_data.mouse + '_' + session_data.date + '_' + 'restructured_data.pkl'
        trial_data = pd.read_pickle(saving_folder + restructured_data_filename)
        dff_trace_filename = session_data.mouse + '_' + session_data.date + '_' + 'smoothed_signal.npy'
        dff = np.load(saving_folder + dff_trace_filename)

        fiber_options = np.array(['left', 'right'])
        fiber_side_numeric = (np.where(fiber_options == session_data.fiber_side)[0] + 1)[0]
        contra_fiber_side_numeric = (np.where(fiber_options != session_data.fiber_side)[0] + 1)[0]

        self.ipsi_data = ZScoredTraces(trial_data, dff, params, fiber_side_numeric, 0)
        self.ipsi_data.get_peaks()
        self.contra_data = ZScoredTraces(trial_data, dff,params, contra_fiber_side_numeric, 0)
        self.contra_data.get_peaks()

In [5]:
mouse_id = 'SNL_photo70'
date = '20220408'
all_experiments = get_all_experimental_records()
experiment_to_process = all_experiments[(all_experiments['date'] == date) & (all_experiments['mouse_id'] == mouse_id)]
session_data = open_experiment(experiment_to_process)

In [6]:
saving_folder = 'T:\\photometry_2AC\\processed_data\\' + mouse_id + '\\'
#saving_folder = 'C:\\Users\\francescag\\Documents\\PhD_Project\\SNL_photo_photometry\\processed_data' + session_data.mouse + '\\'
restructured_data_filename = mouse_id + '_' + date + '_' + 'restructured_data.pkl'
trial_data = pd.read_pickle(saving_folder + restructured_data_filename)

In [7]:
cot_trials = trial_data[trial_data['Sound type'] == 0]

In [8]:
silence_trials = trial_data[trial_data['Sound type'] == 1]

In [9]:
num_silence_trials = silence_trials.shape[0]
per_missed_trials = silence_trials[silence_trials['Trial outcome']==3].shape[0]/num_silence_trials
per_correct_trials = silence_trials[silence_trials['Trial outcome']==1].shape[0]/num_silence_trials
per_incorrect_trials = silence_trials[silence_trials['Trial outcome']==0].shape[0]/num_silence_trials
silence_data_behavioural = pd.DataFrame({'mouse': [mouse_id], 'missed': [per_missed_trials], 'correct':[per_correct_trials], 'incorrect': [per_incorrect_trials], 'stimulus': ['silence']})

In [10]:
num_cot_trials_trials = cot_trials.shape[0]
per_missed_trials = cot_trials[cot_trials['Trial outcome']==3].shape[0]/num_cot_trials_trials
per_correct_trials = cot_trials[cot_trials['Trial outcome']==1].shape[0]/num_cot_trials_trials
per_incorrect_trials = cot_trials[cot_trials['Trial outcome']==0].shape[0]/num_cot_trials_trials
cot_data_behavioural = pd.DataFrame({'mouse': [mouse_id], 'missed': [per_missed_trials], 'correct':[per_correct_trials], 'incorrect': [per_incorrect_trials], 'stimulus': ['tone']})

In [11]:
behavioural_data0 = pd.concat([silence_data_behavioural, cot_data_behavioural]).reset_index(drop=True)

In [12]:
params = {'state_type_of_interest': 5,
    'outcome': 1,
    'last_outcome': 0,  # NOT USED CURRENTLY
    'no_repeats' : 1,
    'last_response': 0,
    'align_to' : 'Time start',
    'instance': 0,
    'plot_range': [-6, 6],
    'first_choice_correct': 0,
    'cue': None}
test = CustomAlignedData(session_data, params)

(5095, 18)
(5095, 18)


In [13]:
_, cot_inds, _ = np.intersect1d(test.contra_data.trial_nums, cot_trials['Trial num'], return_indices=True)
_, silence_inds, _ = np.intersect1d(test.contra_data.trial_nums, silence_trials['Trial num'], return_indices=True)

In [14]:
peaks = test.contra_data.trial_peaks
cot_peaks = [peaks[trial] for trial in cot_inds if peaks[trial].size > 0]
silence_peaks = [peaks[trial] for trial in silence_inds  if peaks[trial].size > 0]

In [15]:
cot_traces = test.contra_data.sorted_traces[cot_inds, :]
silence_traces = test.contra_data.sorted_traces[silence_inds, :]

In [16]:
mean_cot_trace = np.mean(test.contra_data.sorted_traces[cot_inds, :], axis=0)
cot_error_bar_lower, cot_error_bar_upper = calculate_error_bars(mean_cot_trace,
                                                        cot_traces,
                                                        error_bar_method='sem')
                 
mean_silence_trace = np.mean(test.contra_data.sorted_traces[silence_inds, :], axis=0)
silence_error_bar_lower, silence_error_bar_upper = calculate_error_bars(mean_silence_trace,
                                                        silence_traces,
                                                        error_bar_method='sem')
                 

In [17]:
mouse_silence_data = {'mouse': mouse_id, 'tone': [mean_cot_trace], 'tone error bar lower': [cot_error_bar_lower], 'tone error  bar upper': [cot_error_bar_upper], 
                     'silence': [mean_silence_trace], 'silence error bar lower': [silence_error_bar_lower], 'silence error bar upper': [silence_error_bar_upper], 
                     'mean tone peak': np.mean(cot_peaks), 'mean silence peak': np.mean(silence_peaks)}

In [18]:
mouse_silence_df = pd.DataFrame(mouse_silence_data)
all_mouse_silence_df = mouse_silence_df

In [19]:
mouse_id = 'SNL_photo43'
date = '20210603'

all_experiments = get_all_experimental_records()
experiment_to_process = all_experiments[(all_experiments['date'] == date) & (all_experiments['mouse_id'] == mouse_id)]
session_data = open_experiment(experiment_to_process)
fiber_options = ['left', 'right']
fiber_side = experiment_to_process['fiber_side'].values[0]
contra_choice_ind = np.where(np.array(fiber_options) != fiber_side)[0][0] + 1


In [20]:
def get_trial_data(session_data):
    saving_folder = 'T:\\photometry_2AC\\processed_data\\' + session_data.mouse + '\\'
    restructured_data_filename = session_data.mouse + '_' + session_data.date + '_' + 'restructured_data.pkl'
    trial_data = pd.read_pickle(saving_folder + restructured_data_filename)
    return trial_data

In [21]:
trial_data = get_trial_data(session_data)

In [22]:
post_change_trials = trial_data[(trial_data['Trial num']>149) & (trial_data['State type'] == 1)]
silence_correct_trials =  post_change_trials[(post_change_trials['Response'] == contra_choice_ind) & (post_change_trials['Trial outcome'] == 1)] 
silence_trial_type = silence_correct_trials['Trial type'].unique()[0]
silence_trials = post_change_trials[post_change_trials['Trial type'] == silence_trial_type]
cot_trials = trial_data[(trial_data['Trial num']<=149) & (trial_data['State type'] == 1)]

In [23]:
num_silence_trials = silence_trials.shape[0]
per_missed_trials = silence_trials[silence_trials['Trial outcome']==3].shape[0]/num_silence_trials
per_correct_trials = silence_trials[silence_trials['Trial outcome']==1].shape[0]/num_silence_trials
per_incorrect_trials = silence_trials[silence_trials['Trial outcome']==0].shape[0]/num_silence_trials
silence_data_behavioural = pd.DataFrame({'mouse': [mouse_id], 'missed': [per_missed_trials], 'correct':[per_correct_trials], 'incorrect': [per_incorrect_trials], 'stimulus': ['silence']})

In [24]:
num_cot_trials_trials = cot_trials.shape[0]
per_missed_trials = cot_trials[cot_trials['Trial outcome']==3].shape[0]/num_cot_trials_trials
per_correct_trials = cot_trials[cot_trials['Trial outcome']==1].shape[0]/num_cot_trials_trials
per_incorrect_trials = cot_trials[cot_trials['Trial outcome']==0].shape[0]/num_cot_trials_trials
cot_data_behavioural = pd.DataFrame({'mouse': [mouse_id], 'missed': [per_missed_trials], 'correct':[per_correct_trials], 'incorrect': [per_incorrect_trials], 'stimulus': ['tone']})

In [25]:
behavioural_data1 = pd.concat([silence_data_behavioural, cot_data_behavioural]).reset_index(drop=True)

In [26]:
behavioural_data = pd.concat([behavioural_data0, behavioural_data1])

In [27]:
behavioural_data

,mouse,missed,correct,incorrect,stimulus
0,SNL_photo70,0.00000,0.611601,0.388399,silence
1,SNL_photo70,0.00000,0.789445,0.210555,tone
0,SNL_photo43,0.01227,0.957055,0.030675,silence
1,SNL_photo43,0.00000,0.926667,0.073333,tone


In [28]:
params = {'state_type_of_interest': 5,
    'outcome': 1,
    'last_outcome': 0,  # NOT USED CURRENTLY
    'no_repeats' : 1,
    'last_response': 0,
    'align_to' : 'Time start',
    'instance': 0,
    'plot_range': [-6, 6],
    'first_choice_correct': 0,
    'cue': None}
test = CustomAlignedData(session_data, params)
cot_inds = np.where(test.contra_data.trial_nums <=149)[0]
silence_inds = np.where(test.contra_data.trial_nums >149)[0]
cot_traces = test.contra_data.sorted_traces[cot_inds, :]
silence_traces = test.contra_data.sorted_traces[silence_inds, :]
peaks = test.contra_data.trial_peaks
cot_peaks = [peaks[trial] for trial in cot_inds if peaks[trial].size > 0]
silence_peaks = [peaks[trial] for trial in silence_inds  if peaks[trial].size > 0]
mean_cot_trace = np.mean(test.contra_data.sorted_traces[cot_inds, :], axis=0)
cot_error_bar_lower, cot_error_bar_upper = calculate_error_bars(mean_cot_trace,
                                                        cot_traces,
                                                        error_bar_method='sem')
                 
mean_silence_trace = np.mean(test.contra_data.sorted_traces[silence_inds, :], axis=0)
silence_error_bar_lower, silence_error_bar_upper = calculate_error_bars(mean_silence_trace,
                                                        silence_traces,
                                                        error_bar_method='sem')
mouse_silence_data = {'mouse': mouse_id, 'tone': [mean_cot_trace], 'tone error bar lower': [cot_error_bar_lower], 'tone error  bar upper': [cot_error_bar_upper], 
                     'silence': [mean_silence_trace], 'silence error bar lower': [silence_error_bar_lower], 'silence error bar upper': [silence_error_bar_upper], 
                     'mean tone peak': np.mean(cot_peaks), 'mean silence peak': np.mean(silence_peaks)}

(5730, 17)
(5730, 17)


In [29]:
mouse_silence_df = pd.DataFrame(mouse_silence_data)

In [30]:
all_mouse_silence_df = pd.concat([all_mouse_silence_df, mouse_silence_df])

In [31]:
all_mouse_silence_df

,mouse,tone,tone error bar lower,tone error bar upper,silence,silence error bar lower,silence error bar upper,mean tone peak,mean silence peak
0,SNL_photo70,"[0.323979892287553, 0.32390756566394446, 0.323...","[0.19396445647993743, 0.19390492822560362, 0.1...","[0.4539953280951685, 0.4539102031022853, 0.453...","[0.12205941570870465, 0.1221071823232717, 0.12...","[0.0034222032018319526, 0.0034841321110106876,...","[0.24069662821557736, 0.24073023253553272, 0.2...",1.348320,1.137857
0,SNL_photo43,"[0.04908578924340603, 0.049037602641820964, 0....","[-0.051768274351719996, -0.05182109124293623, ...","[0.14993985283853206, 0.14989629652657815, 0.1...","[-0.042925987331389893, -0.042956382395298476,...","[-0.12836532616307567, -0.12838506216320644, -...","[0.042513351500295886, 0.042472297372609505, 0...",0.683896,0.803515


In [32]:
mouse_id = 'SNL_photo37'
date = '20210603'
all_experiments = get_all_experimental_records()
experiment_to_process = all_experiments[(all_experiments['date'] == date) & (all_experiments['mouse_id'] == mouse_id)]
session_data = open_experiment(experiment_to_process)

fiber_options = ['left', 'right']
fiber_side = experiment_to_process['fiber_side'].values[0]
contra_choice_ind = np.where(np.array(fiber_options) != fiber_side)[0][0] + 1


In [33]:
trial_data = get_trial_data(session_data)

post_change_trials = trial_data[(trial_data['Trial num']>149) & (trial_data['State type'] == 1)]
silence_correct_trials =  post_change_trials[(post_change_trials['Response'] == contra_choice_ind) & (post_change_trials['Trial outcome'] == 1)] 
silence_trial_type = silence_correct_trials['Trial type'].unique()[0]
silence_trials = post_change_trials[post_change_trials['Trial type'] == silence_trial_type]
cot_trials = trial_data[(trial_data['Trial num']<=149) & (trial_data['State type'] == 1)]

num_silence_trials = silence_trials.shape[0]
per_missed_trials = silence_trials[silence_trials['Trial outcome']==3].shape[0]/num_silence_trials
per_correct_trials = silence_trials[silence_trials['Trial outcome']==1].shape[0]/num_silence_trials
per_incorrect_trials = silence_trials[silence_trials['Trial outcome']==0].shape[0]/num_silence_trials
silence_data_behavioural = pd.DataFrame({'mouse': [mouse_id], 'missed': [per_missed_trials], 'correct':[per_correct_trials], 'incorrect': [per_incorrect_trials], 'stimulus': ['silence']})

num_cot_trials_trials = cot_trials.shape[0]
per_missed_trials = cot_trials[cot_trials['Trial outcome']==3].shape[0]/num_cot_trials_trials
per_correct_trials = cot_trials[cot_trials['Trial outcome']==1].shape[0]/num_cot_trials_trials
per_incorrect_trials = cot_trials[cot_trials['Trial outcome']==0].shape[0]/num_cot_trials_trials
cot_data_behavioural = pd.DataFrame({'mouse': [mouse_id], 'missed': [per_missed_trials], 'correct':[per_correct_trials], 'incorrect': [per_incorrect_trials], 'stimulus': ['tone']})

In [34]:
behavioural_data1 = pd.concat([silence_data_behavioural, cot_data_behavioural]).reset_index(drop=True)
behavioural_data = pd.concat([behavioural_data, behavioural_data1])

In [35]:
behavioural_data

,mouse,missed,correct,incorrect,stimulus
0,SNL_photo70,0.000000,0.611601,0.388399,silence
1,SNL_photo70,0.000000,0.789445,0.210555,tone
0,SNL_photo43,0.012270,0.957055,0.030675,silence
1,SNL_photo43,0.000000,0.926667,0.073333,tone
0,SNL_photo37,0.011429,0.908571,0.080000,silence
1,SNL_photo37,0.000000,0.826667,0.173333,tone


In [36]:
params = {'state_type_of_interest': 5,
    'outcome': 1,
    'last_outcome': 0,  # NOT USED CURRENTLY
    'no_repeats' : 1,
    'last_response': 0,
    'align_to' : 'Time start',
    'instance': 0,
    'plot_range': [-6, 6],
    'first_choice_correct': 0,
    'cue': None}
test = CustomAlignedData(session_data, params)

cot_inds = np.where(test.contra_data.trial_nums <=149)[0]
silence_inds = np.where(test.contra_data.trial_nums >149)[0]
peaks = test.contra_data.trial_peaks
cot_peaks = [peaks[trial] for trial in cot_inds if peaks[trial].size > 0]
silence_peaks = [peaks[trial] for trial in silence_inds  if peaks[trial].size > 0]

cot_traces = test.contra_data.sorted_traces[cot_inds, :]
silence_traces = test.contra_data.sorted_traces[silence_inds, :]
mean_cot_trace = np.mean(test.contra_data.sorted_traces[cot_inds, :], axis=0)
cot_error_bar_lower, cot_error_bar_upper = calculate_error_bars(mean_cot_trace,
                                                        cot_traces,
                                                        error_bar_method='sem')
                 
mean_silence_trace = np.mean(test.contra_data.sorted_traces[silence_inds, :], axis=0)
silence_error_bar_lower, silence_error_bar_upper = calculate_error_bars(mean_silence_trace,
                                                        silence_traces,
                                                        error_bar_method='sem')
mouse_silence_data = {'mouse': mouse_id, 'tone': [mean_cot_trace], 'tone error bar lower': [cot_error_bar_lower], 'tone error  bar upper': [cot_error_bar_upper], 
                     'silence': [mean_silence_trace], 'silence error bar lower': [silence_error_bar_lower], 'silence error bar upper': [silence_error_bar_upper], 
                     'mean tone peak': np.mean(cot_peaks), 'mean silence peak': np.mean(silence_peaks)}

(5178, 17)
(5178, 17)


In [37]:
mouse_silence_df = pd.DataFrame(mouse_silence_data)

In [38]:
all_mouse_silence_df = pd.concat([all_mouse_silence_df, mouse_silence_df])

In [39]:
all_mouse_silence_df = all_mouse_silence_df.reset_index(drop=True)

In [40]:
all_mouse_silence_df

,mouse,tone,tone error bar lower,tone error bar upper,silence,silence error bar lower,silence error bar upper,mean tone peak,mean silence peak
0,SNL_photo70,"[0.323979892287553, 0.32390756566394446, 0.323...","[0.19396445647993743, 0.19390492822560362, 0.1...","[0.4539953280951685, 0.4539102031022853, 0.453...","[0.12205941570870465, 0.1221071823232717, 0.12...","[0.0034222032018319526, 0.0034841321110106876,...","[0.24069662821557736, 0.24073023253553272, 0.2...",1.348320,1.137857
1,SNL_photo43,"[0.04908578924340603, 0.049037602641820964, 0....","[-0.051768274351719996, -0.05182109124293623, ...","[0.14993985283853206, 0.14989629652657815, 0.1...","[-0.042925987331389893, -0.042956382395298476,...","[-0.12836532616307567, -0.12838506216320644, -...","[0.042513351500295886, 0.042472297372609505, 0...",0.683896,0.803515
2,SNL_photo37,"[-0.08827216780229113, -0.08827726241329434, -...","[-0.22503286724982569, -0.2250359791604583, -0...","[0.048488531645243416, 0.048481454333869634, 0...","[-0.0010413843402279647, -0.001142962026786660...","[-0.08053625297024745, -0.08063092748853151, -...","[0.07845348428979153, 0.07834500343495818, 0.0...",1.080463,0.952484


In [41]:
concatenated_tone_traces = np.array([all_mouse_silence_df['tone'].values[i] for i in all_mouse_silence_df.index.values])

In [42]:
all_time_points = decimate(test.contra_data.time_points, 10)
start_plot = int(all_time_points.shape[0]/2 - 2*1000)
end_plot = int(all_time_points.shape[0]/2 + 2*1000)
time_points = all_time_points[start_plot: end_plot]
mean_cot_traces = decimate(np.mean(concatenated_tone_traces, axis=0), 10)[start_plot: end_plot]

In [43]:
concatenated_silence_traces = np.array([all_mouse_silence_df['silence'].values[i] for i in all_mouse_silence_df.index.values])
mean_silence_traces = decimate(np.mean(concatenated_silence_traces, axis=0), 10)[start_plot: end_plot]

In [44]:
all_mice_silence_error_bar_lower, all_mice_silence_error_bar_upper = calculate_error_bars(mean_silence_traces,
                                                        decimate(concatenated_silence_traces, 10)[:, start_plot: end_plot],
                                                        error_bar_method='sem')
all_mice_cot_error_bar_lower, all_mice_cot_error_bar_upper = calculate_error_bars(mean_cot_traces,
                                                        decimate(concatenated_tone_traces, 10)[:, start_plot: end_plot],
                                                        error_bar_method='sem')               

In [95]:
import matplotlib
import os

In [96]:
font = {'size': 7}
matplotlib.rc('font', **font)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['font.sans-serif'] = 'Arial'
matplotlib.rcParams['font.family']

fig, axs1 = plt.subplots(1,1, figsize=[2.5,2])
colours = cm.viridis(np.linspace(0.2, 0.7, 2))
axs1.plot(time_points, mean_cot_traces, label='normal cue', color=colours[0])
axs1.fill_between(time_points, all_mice_cot_error_bar_lower, all_mice_cot_error_bar_upper, alpha=0.4, linewidth=0, color=colours[0])
axs1.plot(time_points, mean_silence_traces, label='silence', color=colours[1])
axs1.fill_between(time_points, all_mice_silence_error_bar_lower, all_mice_silence_error_bar_upper, alpha=0.4, linewidth=0, color=colours[1])
axs1.set_xlim([-2,2])
axs1.axvline([0], color='k')
axs1.set_ylabel('z-scored fluorescence', fontsize=8)
axs1.set_xlabel('Time (s)', fontsize=8)

axs1.spines['right'].set_visible(False)
axs1.spines['top'].set_visible(False)
axs1.legend(loc='lower left', bbox_to_anchor=(0.7, 0.8),
            borderaxespad=0, frameon=False,prop={'size': 6 })
plt.tight_layout()
#filepath=os.path.join('W:\\paper', 'silence_across_mice_traces.pdf')
#fig.savefig(filepath)

In [67]:
plt.plot(mean_cot_traces, color='b')
plt.plot(mean_silence_traces, color='orange')

In [139]:
concatenated_traces

(3, 160000)

In [144]:
all_mouse_silence_df.index.values

array([0, 1, 2], dtype=int64)

In [177]:
cot_peaks = all_mouse_silence_df['mean tone peak'].values

In [178]:
silence_peaks = all_mouse_silence_df['mean silence peak'].values

In [179]:
from scipy.stats import ttest_rel, shapiro

In [180]:
differences = cot_peaks - silence_peaks
shapiro(differences)

ShapiroResult(statistic=0.9230131506919861, pvalue=0.4630334675312042)

In [181]:
ttest_rel(cot_peaks, silence_peaks)

Ttest_relResult(statistic=0.7354353598866584, pvalue=0.5386253835905292)

In [45]:
df_for_plot_behavioural = behavioural_data.pivot(index='mouse', columns='stimulus', values='missed')

In [46]:
cols = df_for_plot_behavioural.columns.tolist()

In [47]:
cols = cols[-1:] + cols[:-1]

In [48]:
df_for_plot_behavioural = 100 - (df_for_plot_behavioural[cols] *100)

In [49]:
df_for_plot_behavioural

stimulus,tone,silence
mouse,,
SNL_photo37,100.0,98.857143
SNL_photo43,100.0,98.773006
SNL_photo70,100.0,100.000000


In [50]:
np.mean(df_for_plot_behavioural['silence'])

99.2100496640374

In [51]:
np.std(df_for_plot_behavioural['silence'])

0.5596343431027693

In [331]:
differences = df_for_plot_behavioural['tone'].values - df_for_plot_behavioural['silence'].values
shapiro(differences)

ShapiroResult(statistic=0.8011702299118042, pvalue=0.11729390174150467)

In [332]:
ttest_rel(df_for_plot_behavioural['tone'],df_for_plot_behavioural['silence'])

Ttest_relResult(statistic=1.9962293102415072, pvalue=0.18401751379431722)

In [127]:
fig, ax = plt.subplots(figsize=[2,2])
df_for_plot_behavioural.T.plot(ax=ax)

<AxesSubplot:xlabel='stimulus'>

In [333]:
def francescas_plot(ax, data, show_err_bar=False, mean_linewidth=4, mean_line_color='blue', legend=False):
    """Plot lines for all mice from early to late.
    """
    data.plot(ax=ax,  legend=False, color='grey')
    data.mean(1).plot(ax=ax,linewidth=mean_linewidth,color=mean_line_color, alpha=0.7)

    if show_err_bar:
        yerr = data.std(axis=1)

        plt.errorbar(np.array([0,1]), data.mean(1), yerr, color=mean_line_color, linewidth=4)

    # drop the left and bottom spine for cool looking effect
    ax.spines['left'].set_position(('outward', 10))
    ax.spines['bottom'].set_position(('outward', 10))
    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)


In [342]:
peak_size_df = all_mouse_silence_df[['mouse', 'mean tone peak', 'mean silence peak']].set_index('mouse')
font = {'size': 7}
matplotlib.rc('font', **font)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['font.sans-serif'] = 'Arial'
matplotlib.rcParams['font.family']


fig, ax = plt.subplots(figsize=[1.3,2])

francescas_plot(ax, peak_size_df.T, mean_line_color='#7FB5B5', mean_linewidth=0, show_err_bar=False)
plt.xticks([0, 1], ['tone', 'silence'], fontsize=8)
plt.xticks([0, 1], ['tone', 'silence'], fontsize=8)
plt.ylabel('dLight z-score', fontsize=8)
#ax.text(0.9, 1.9, 'p-value = {0:.3f}'.format(pval))
ax.set_xlabel(' ')

#significance stars
y = peak_size_df.T.to_numpy().max() + .2
h = .001

ax.text(.5, y+h, 'n.s.', ha='center', fontsize=8)

plt.tight_layout()

In [341]:
font = {'size': 7}
matplotlib.rc('font', **font)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['font.sans-serif'] = 'Arial'
matplotlib.rcParams['font.family']


fig, ax = plt.subplots(figsize=[1.5,2])

francescas_plot(ax, df_for_plot_behavioural.T, mean_line_color='#7FB5B5', mean_linewidth=0, show_err_bar=False)
plt.xticks([0, 1], ['tone', 'silence'], fontsize=8)
plt.ylabel('Missed trials (%)', fontsize=8)
#ax.text(0.9, 1.9, 'p-value = {0:.3f}'.format(pval))
ax.set_xlabel(' ')

#significance stars
y = df_for_plot_behavioural.T.to_numpy().max() + .2
h = .1
plt.plot([0, 0, 1, 1], [y, y+h, y+h, y],c='k',lw=1)
ax.text(.5, y+h, 'n.s.', ha='center', fontsize=8)
ax.set_ylim([95, 100])
plt.tight_layout()
#filepath=os.path.join('W:\\paper', 'silence_across_completed_trials_mice_peaks_with_stats.pdf')
#fig.savefig(filepath)

In [218]:
example_mouse_data = all_mouse_silence_df[all_mouse_silence_df['mouse']== 'SNL_photo70']

In [219]:
example_mouse_data['tone'].values

array([array([0.04908579, 0.0490376 , 0.04898933, ..., 0.060728  , 0.0607879 ,
              0.06084786])                                                    ],
      dtype=object)

In [220]:
all_time_points = decimate(test.contra_data.time_points, 10)
start_plot = int(all_time_points.shape[0]/2 - 2*1000)
end_plot = int(all_time_points.shape[0]/2 + 2*1000)
time_points = all_time_points[start_plot: end_plot]

In [221]:
example_tone_mean_trace = decimate(example_mouse_data['tone'][0], 10)[start_plot:end_plot]
example_tone_upper_error = decimate(example_mouse_data['tone error  bar upper'][0], 10)[start_plot:end_plot]
example_tone_lower_error = decimate(example_mouse_data['tone error bar lower'][0], 10)[start_plot:end_plot]

In [222]:
example_silence_mean_trace = decimate(example_mouse_data['silence'][0], 10)[start_plot:end_plot]
example_silence_upper_error = decimate(example_mouse_data['silence error bar upper'][0], 10)[start_plot:end_plot]
example_silence_lower_error = decimate(example_mouse_data['silence error bar lower'][0], 10)[start_plot:end_plot]

In [223]:
font = {'size': 7}
matplotlib.rc('font', **font)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['font.sans-serif'] = 'Arial'
matplotlib.rcParams['font.family']

fig, axs1 = plt.subplots(1,1, figsize=[2.5,2])
colours = cm.viridis(np.linspace(0.2, 0.7, 2))
axs1.plot(time_points, example_tone_mean_trace, label='normal cue', color=colours[0])
axs1.fill_between(time_points, example_tone_lower_error, example_tone_upper_error, alpha=0.4, linewidth=0, color=colours[0])
axs1.plot(time_points, example_silence_mean_trace, label='silence', color=colours[1])
axs1.fill_between(time_points, example_silence_lower_error, example_silence_upper_error, alpha=0.4, linewidth=0, color=colours[1])
axs1.set_xlim([-2,2])
axs1.axvline([0], color='k')
axs1.set_ylabel('z-scored fluorescence', fontsize=8)
axs1.set_xlabel('Time (s)', fontsize=8)

axs1.spines['right'].set_visible(False)
axs1.spines['top'].set_visible(False)
axs1.legend(loc='lower left', bbox_to_anchor=(0.7, 0.8),
            borderaxespad=0, frameon=False,prop={'size': 6 })
plt.tight_layout()
#filepath=os.path.join('W:\\paper', 'silence_example_traces.pdf')
#fig.savefig(filepath)

In [ ]:
df_for_plot_behaviour = beha[['mouse', 'mean tone peak', 'mean silence peak']].set_index('mouse').T

In [31]:
#plot behaviour
font = {'size': 7}
matplotlib.rc('font', **font)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['font.sans-serif'] = 'Arial'
matplotlib.rcParams['font.family']


fig, ax = plt.subplots(figsize=[2,2])
francescas_plot(ax, df_for_plot_behaviour, mean_line_color='#7FB5B5', mean_linewidth=0, show_err_bar=False)
plt.xticks([0, 1], ['tone', 'silence'], fontsize=8)
plt.ylabel('Z-scored fluorescence', fontsize=8)
#ax.text(0.9, 1.9, 'p-value = {0:.3f}'.format(pval))
ax.set_xlabel(' ')

#significance stars
y = df_for_plot.to_numpy().max() + .2
h = .1
plt.plot([0, 0, 1, 1], [y, y+h, y+h, y],c='k',lw=1)
#ax.text(.5, y+h, 'n.s.', ha='center', fontsize=10)

plt.tight_layout()
filepath=os.path.join('W:\\paper', 'behvaiour_silence_with_stats.pdf')
fig.savefig(filepath)

,mouse,tone,tone error bar lower,tone error bar upper,silence,silence error bar lower,silence error bar upper,mean tone peak,mean silence peak
0,SNL_photo70,"[0.24152475728319503, 0.24142043117963471, 0.2...","[0.14148073120054194, 0.14138743696525122, 0.1...","[0.3415687833658481, 0.34145342539401824, 0.34...","[0.12205941570870465, 0.1221071823232717, 0.12...","[0.0034222032018319526, 0.0034841321110106876,...","[0.24069662821557736, 0.24073023253553272, 0.2...",1.383224,1.137857
1,SNL_photo43,"[-0.006365803904310347, -0.006389817225110093,...","[-0.10292189600644441, -0.10294199891928427, -...","[0.09019028819782371, 0.09016236446906409, 0.0...","[-0.05814355103413557, -0.058146821296290147, ...","[-0.13443391428646181, -0.13442747556905332, -...","[0.018146812218190672, 0.018133832976473016, 0...",0.697018,0.755536
2,SNL_photo37,"[-0.08769776420871503, -0.08766363155609597, -...","[-0.21983474416973495, -0.2197982979795826, -0...","[0.04443921575230489, 0.04447103486739068, 0.0...","[-0.08063631094017584, -0.08071873095022479, -...","[-0.15440428764626124, -0.15448078478012428, -...","[-0.006868334234090431, -0.006956677120325286,...",1.152473,0.953373
